In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount='true')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import os
import gdown
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

In [ ]:
! unzip /content/drive/MyDrive/nhats_clock.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: class/train/4/13312.png  
  inflating: class/train/4/13313.png  
  inflating: class/train/4/13315.png  
  inflating: class/train/4/13318.png  
  inflating: class/train/4/13322.png  
  inflating: class/train/4/13328.png  
  inflating: class/train/4/1333.png  
  inflating: class/train/4/13331.png  
  inflating: class/train/4/13332.png  
  inflating: class/train/4/1334.png  
  inflating: class/train/4/13342.png  
  inflating: class/train/4/13343.png  
  inflating: class/train/4/13347.png  
  inflating: class/train/4/13356.png  
  inflating: class/train/4/13357.png  
  inflating: class/train/4/13358.png  
  inflating: class/train/4/1336.png  
  inflating: class/train/4/13362.png  
  inflating: class/train/4/1337.png  
  inflating: class/train/4/13374.png  
  inflating: class/train/4/13378.png  
  inflating: class/train/4/13379.png  
  inflating: class/train/4/13380.png  
  inflating: class/train/4/13389.png  
  inflating: class/train/4/1339.png  
  inflati

In [ ]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(root='/content/class/train')
test_dataset = ImageFolder(root='/content/class/test')

print("train_dataset:", len(train_dataset))
print("test_dataset:", len(test_dataset))

train_dataset: 17400
test_dataset: 600


In [ ]:
from torch.utils.data import random_split

n_samples = len(train_dataset)
train_size = int(n_samples * 0.8)
valid_size = n_samples - train_size

train_dataset, valid_dataset = random_split(train_dataset, [train_size, valid_size])

print("train data", len(train_dataset))
print("validation data", len(valid_dataset))

train data 13920
validation data 3480


In [ ]:
from torchvision import transforms
train_transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

valid_transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [ ]:
TRAINDATA_PATH = '/content/class/train'

label_name = {"0": 0, "1": 1, "2": 2 , "3": 3, "4": 4, "5": 5}
def get_img_info(data_dir):
        imgpath = []
        imglabel = []
        for root, dirs, _ in os.walk(data_dir):
            for sub_dir in dirs:
                img_names = os.listdir(os.path.join(root, sub_dir))
                img_names = list(filter(lambda x: x.endswith('.png'), img_names))

                for i in range(len(img_names)):
                    img_name = img_names[i]
                    path_img = os.path.join(root, sub_dir, img_name)
                    label = label_name[sub_dir]
                    imgpath.append(path_img)
                    imglabel.append(int(label))

        return imgpath,  imglabel

imgpath,  imglabel = get_img_info(TRAINDATA_PATH)

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self,trainData,trainLabel,transform=None):

        self.images = trainData
        self.label = trainLabel
        self.transform = transform

    def __getitem__(self, index):

        imgpath = self.images[index]
        img = Image.open(imgpath).convert('RGB')

        label = self.label[index]

        if self.transform:
          img = self.transform(img)

        return img, label

    def __len__(self):

        return len(self.images)

In [ ]:
train_img, val_img, train_label, val_label = train_test_split(imgpath, imglabel, test_size=0.2, random_state=42)

train_set = Custom_dataset(train_img, train_label, transform=train_transform)
valid_set = Custom_dataset(val_img, val_label, transform=valid_transform)

print('Number of total training data:', len(train_set))
print('Number of total validation data:', len(valid_set))

class_num = 6
classes = ('0', '1', '2', '3', '4', '5')

Number of total training data: 13920
Number of total validation data: 3480


In [ ]:
from torchvision import transforms

test_dataset = ImageFolder(root="/content/class/test",
                           transform=valid_transform
                      )

In [ ]:
from torch.utils.data import DataLoader

trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
validloader = DataLoader(valid_set, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

In [ ]:
import torch.nn as nn
import torchvision.models as models

# define a class named "Model"
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # Load the ResNet-50 model
        resnet = models.resnet50(pretrained=False)

        # Remove the fully connected layers at the end
        resnet = nn.Sequential(*list(resnet.children())[:-2])

        # Add it to your model
        self.resnet = resnet

        # Define your own classifier
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(0.7),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 6)
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)
        return x

model = Model()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# Load the pretrained weights
pretrained_weight_path = "/content/drive/MyDrive/simclr_pretrained_weight.pth"
pretrained_weights = torch.load(pretrained_weight_path)
model.load_state_dict(pretrained_weights, strict=False)

_IncompatibleKeys(missing_keys=['resnet.0.weight', 'resnet.1.weight', 'resnet.1.bias', 'resnet.1.running_mean', 'resnet.1.running_var', 'resnet.4.0.conv1.weight', 'resnet.4.0.bn1.weight', 'resnet.4.0.bn1.bias', 'resnet.4.0.bn1.running_mean', 'resnet.4.0.bn1.running_var', 'resnet.4.0.conv2.weight', 'resnet.4.0.bn2.weight', 'resnet.4.0.bn2.bias', 'resnet.4.0.bn2.running_mean', 'resnet.4.0.bn2.running_var', 'resnet.4.0.conv3.weight', 'resnet.4.0.bn3.weight', 'resnet.4.0.bn3.bias', 'resnet.4.0.bn3.running_mean', 'resnet.4.0.bn3.running_var', 'resnet.4.0.downsample.0.weight', 'resnet.4.0.downsample.1.weight', 'resnet.4.0.downsample.1.bias', 'resnet.4.0.downsample.1.running_mean', 'resnet.4.0.downsample.1.running_var', 'resnet.4.1.conv1.weight', 'resnet.4.1.bn1.weight', 'resnet.4.1.bn1.bias', 'resnet.4.1.bn1.running_mean', 'resnet.4.1.bn1.running_var', 'resnet.4.1.conv2.weight', 'resnet.4.1.bn2.weight', 'resnet.4.1.bn2.bias', 'resnet.4.1.bn2.running_mean', 'resnet.4.1.bn2.running_var', 'resn

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler

torch.cuda.empty_cache()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
model = model.to(device)  # Move the model to the GPU

optimizer = optim.Adam(model.parameters(), lr=1E-3)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
loss_func = nn.CrossEntropyLoss()
epochs = 50

# create empty lists for saving metrics during training
train_loss_list = []
train_accuracy_list = []
valid_loss_list = []
valid_accuracy_list = []

for epoch in range(epochs):

    print(f"Epoch {epoch+1}/{epochs}")

    # initialize metrics value
    train_correct_count = 0
    train_accuracy = 0
    train_loss = 0
    valid_correct_count = 0
    valid_accuracy = 0
    valid_loss = 0

    #--- Training Phase ---#
    model.train()    # set model to training mode

    pbar = tqdm(trainloader)
    pbar.set_description("Train")

    for x_batch, y_batch in pbar:      # take mini batch data from train_loader

        x_batch = x_batch.to(device)     # load x_batch data on GPU
        y_batch = y_batch.to(device)     # load y_batch data on GPU

        optimizer.zero_grad()                  # reset gradients to 0
        p_batch = model(x_batch)               # do prediction
        loss = loss_func(p_batch, y_batch)     # measure loss
        loss.backward()                        # calculate gradients
        optimizer.step()                       # update model parameters
        scheduler.step()

        train_loss += loss.item()                                # accumulate loss value
        p_batch_label = torch.argmax(p_batch, dim=1)             # convert p_batch vector to p_batch_label
        train_correct_count += (p_batch_label == y_batch).sum()  # count up number of correct predictions

        pbar.set_postfix({"accuracy":f"{(p_batch_label == y_batch).sum()/len(x_batch):.4f}", "loss": f"{loss.item():.4f}"})
    #----------------------#

    #--- Evaluation Phase ---#
    with torch.no_grad():   # disable autograd for saving memory usage
        model.eval()        # set model to evaluation mode

        pbar = tqdm(validloader)
        pbar.set_description("Valid")

        for x_batch, y_batch in pbar:   # take mini batch data from test_loader

            x_batch = x_batch.to(device)     # load x_batch data on GPU
            y_batch = y_batch.to(device)     # load y_batch data on GPU

            p_batch = model(x_batch)              # do prediction
            loss = loss_func(p_batch, y_batch)    # measure loss

            valid_loss += loss.item()                                # accumulate loss value
            p_batch_label = torch.argmax(p_batch, dim=1)            # convert p_batch vector to p_batch_label
            valid_correct_count += (p_batch_label == y_batch).sum()  # count up number of correct predictions

            pbar.set_postfix({"accuracy":f"{(p_batch_label == y_batch).sum()/len(x_batch):.4f}", "loss": f"{loss.item():.4f}"})
    #------------------------#

    train_accuracy = train_correct_count.item()/len(train_dataset)   # determine accuracy for training data
    valid_accuracy = valid_correct_count.item()/len(valid_dataset)   # determin accuracy for test data
    train_loss = train_loss/len(trainloader)       # determin loss for training data
    valid_loss = valid_loss/len(validloader)       # determin loss for validation data

    # show and store metrics
    print(f"Train: Accuracy={train_accuracy:.3f} Loss={train_loss:.3f}, Valid: Accuracy={valid_accuracy:.3f} Loss={valid_loss:.3f}")
    train_accuracy_list.append(train_accuracy)
    train_loss_list.append(train_loss)
    valid_accuracy_list.append(valid_accuracy)
    valid_loss_list.append(valid_loss)

    print("---------------------")

cuda:0
Epoch 1/50


  0%|          | 0/109 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


  0%|          | 0/28 [00:00<?, ?it/s]

Train: Accuracy=0.175 Loss=1.892, Valid: Accuracy=0.157 Loss=1.793
---------------------
Epoch 2/50


...

Train: Accuracy=0.180 Loss=1.822, Valid: Accuracy=0.160 Loss=1.793
---------------------
Epoch 13/50


  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

Train: Accuracy=0.181 Loss=1.821, Valid: Accuracy=0.159 Loss=1.793
---------------------
Epoch 14/50


  0%|          | 0/109 [00:00<?, ?it/s]

In [ ]:
plt.figure(figsize=(5, 3))
plt.rcParams["font.size"] = 14
plt.plot(np.arange(epochs) + 1, train_accuracy_list, label="train")
plt.plot(np.arange(epochs) + 1, valid_accuracy_list, label="valid")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(np.arange(epochs)+1, train_loss_list, label="train")
plt.plot(np.arange(epochs)+1, valid_loss_list, label="valid")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.legend()

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 讀取 CSV 檔案
df = pd.read_csv('a.csv')

# 提取真實標籤和預測標籤
true_labels = df['TRUE']
predicted_labels = df['predict']

conf_matrix = confusion_matrix(true_labels, predicted_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Y_pred')
plt.ylabel('Y_test')
plt.title('confusion matrix')
plt.show()